# Training the Convolutional Model

## Pre-Existing Code: Building the Data

In [22]:
import os
import cv2
import numpy as np
from tqdm import tqdm


REBUILD_DATA = False # set to true to one once, then back to false unless you want to change something in your training data.

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        # iterating over all the directories
        for label in self.LABELS:
            print(label)
            # iterate over all the jpg images in the directories
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        # some of the images are no good
                        # some might be corrupt or empty
                        # normally would print(str(e)) but we already know what the error is
                        pass
                        #print(label, f, str(e))

        # after we run the for loop, shuffle data
        # shuffles in-place so we dont have to reassign data
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()


training_data = np.load("training_data.npy", allow_pickle=True) # dunno whats up with allow_pickle, but need it to load data in this case
print(len(training_data))

24946


## Our Convolutional Model

- 3 convolutional layers
- layers are 2d, for image processing
>
- at some point we have to have a 2 node linear layer for Dog v Cat
- getting to this is difficult
- tensorflow has a flatten function, torch does not
>

## Flattening Data
 
> Transitioning from convolutional to linear layers

- how do we get an input number when transitioning from fully connected Convolutional layers to fully connected linear layers?
- [vid reference](https://www.youtube.com/watch?v=1gQR24B3ISE&t=473s)

In [23]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 



class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)
        #self.fc1 = nn.Linear(????, 512)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        # in the initialization of calling the layers, 
        # we have to do a quick forward pass to see what the input number is
        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        print(x[0].shape)

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        # run forward pass again to determine number
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #return x
        
        # probably need some sort of activation layer, but thats not really required
        # just a bunch of vectors being multiplied by each other
        # probably should just have one
        
        # Neural Networks are really just trial and error
        return F.softmax(x, dim=1)

net = Net()

torch.Size([128, 2, 2])


**^Nueral Networks are just trail and error!!!**

## Passing Data through our model

In [24]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0 # scale imagery from o,255 to be between 0,1

y = torch.Tensor([i[1] for i in training_data])

# separate out some training data

VAL_PCT = 0.1 # test against 10% of out data
val_size = int(len(X)*VAL_PCT)
print(val_size) 

2494


In [32]:
# slicing data into training and testing data

# train up to the negative val_size
train_X = X[:-val_size]
train_y = y[:-val_size]

# train on what is past the negative val_size
test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [33]:
# quickest and easiest thing to change if hitting mem errors is the batch size
# dont want to go down to one
# go down to 8, and then start tweaking the model

BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        #print(i, i+BATCH_SIZE)
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad() # function to zero the gradients depends (see below)
        outputs = net(batch_X) # now we can calculate loss
        
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step() # now we can train the model
        
    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|          | 1/225 [00:00<00:23,  9.55it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])


  2%|▏         | 5/225 [00:00<00:19, 11.02it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])


  3%|▎         | 7/225 [00:00<00:19, 11.10it/s]

...

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])


 98%|█████████▊| 221/225 [00:18<00:00, 11.66it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])


100%|██████████| 225/225 [00:19<00:00, 11.69it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
Epoch: 0. Loss: 0.14699016511440277


**^ [explantation of ouput](https://www.youtube.com/watch?v=1gQR24B3ISE&t=1657s)**

## Fitment & Optimization

 - zero the gradients 
 - ^ [link to explanation](https://www.youtube.com/watch?v=1gQR24B3ISE&t=1736s)

In [34]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        print(real_class, net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

  5%|▌         | 127/2494 [00:00<00:03, 640.54it/s]

torch.Size([128, 2, 2])
tensor(0) tensor([0.6930, 0.3070])
torch.Size([128, 2, 2])
tensor(1) tensor([0.9704, 0.0296])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0841, 0.9159])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0507, 0.9493])
torch.Size([128, 2, 2])
tensor(0) tensor([0.4178, 0.5822])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5773, 0.4227])
torch.Size([128, 2, 2])
tensor(0) tensor([0.8807, 0.1193])
torch.Size([128, 2, 2])
tensor(1) tensor([0.3491, 0.6509])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1815, 0.8185])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1232, 0.8768])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1579, 0.8421])
torch.Size([128, 2, 2])
tensor(1) tensor([0.4449, 0.5551])
torch.Size([128, 2, 2])
tensor(1) tensor([0.3957, 0.6043])
torch.Size([128, 2, 2])
tensor(0) tensor([0.8289, 0.1711])
torch.Size([128, 2, 2])
tensor(1) tensor([0.3584, 0.6416])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1484, 0.8516])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6769, 0.3231

 11%|█         | 277/2494 [00:00<00:03, 721.02it/s]

tensor(1) tensor([0.1066, 0.8934])
torch.Size([128, 2, 2])
tensor(0) tensor([0.2830, 0.7170])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6927, 0.3073])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0155, 0.9845])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0641, 0.9359])
torch.Size([128, 2, 2])
tensor(0) tensor([0.9091, 0.0909])
torch.Size([128, 2, 2])
tensor(0) tensor([0.9241, 0.0759])
torch.Size([128, 2, 2])
tensor(1) tensor([0.2446, 0.7554])
torch.Size([128, 2, 2])
tensor(1) tensor([0.7213, 0.2787])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1854, 0.8146])
torch.Size([128, 2, 2])
tensor(0) tensor([0.4200, 0.5800])
torch.Size([128, 2, 2])
tensor(0) tensor([0.2577, 0.7423])
torch.Size([128, 2, 2])
tensor(0) tensor([0.8157, 0.1843])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0770, 0.9230])
torch.Size([128, 2, 2])
tensor(0) tensor([0.9586, 0.0414])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1240, 0.8760])
torch.Size([128, 2, 2])
tensor(0) tensor([0.4438, 0.5562])
torch.Size([128, 2, 2

 17%|█▋        | 435/2494 [00:00<00:02, 753.53it/s]

...

torch.Size([128, 2, 2])
tensor(0) tensor([0.1127, 0.8873])
torch.Size([128, 2, 2])
tensor(1) tensor([0.8235, 0.1765])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6025, 0.3975])
torch.Size([128, 2, 2])
tensor(1) tensor([0.7583, 0.2417])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5186, 0.4814])
torch.Size([128, 2, 2])
tensor(0) tensor([0.9644, 0.0356])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0320, 0.9680])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5216, 0.4784])
torch.Size([128, 2, 2])
tensor(1) tensor([0.2531, 0.7469])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5804, 0.4196])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0526, 0.9474])
torch.Size([128, 2, 2])
tensor(1) tensor([0.7887, 0.2113])
torch.Size([128, 2, 2])
tensor(1) tensor([0.2355, 0.7645])
torch.Size([128, 2, 2])
tensor(0) tensor([0.3100, 0.6900])
torch.Size([128, 2, 2])
tensor(0) tensor([0.2636, 0.7364])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1192, 0.8808])
torch.Size([128, 2, 2])
tensor(0) tensor([0.6697, 0.3303

 96%|█████████▋| 2403/2494 [00:03<00:00, 714.58it/s]

torch.Size([128, 2, 2])
tensor(1) tensor([0.4618, 0.5382])
torch.Size([128, 2, 2])
tensor(1) tensor([0.4134, 0.5866])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5177, 0.4823])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6676, 0.3324])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6991, 0.3009])
torch.Size([128, 2, 2])
tensor(1) tensor([0.3690, 0.6310])
torch.Size([128, 2, 2])
tensor(0) tensor([0.4371, 0.5629])
torch.Size([128, 2, 2])
tensor(1) tensor([0.7185, 0.2815])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1163, 0.8837])
torch.Size([128, 2, 2])
tensor(1) tensor([0.2840, 0.7160])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1198, 0.8802])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1685, 0.8315])
torch.Size([128, 2, 2])
tensor(0) tensor([0.4571, 0.5429])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0838, 0.9162])
torch.Size([128, 2, 2])
tensor(0) tensor([0.4660, 0.5340])
torch.Size([128, 2, 2])
tensor(0) tensor([0.3435, 0.6565])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6609, 0.3391

100%|██████████| 2494/2494 [00:03<00:00, 704.34it/s]

torch.Size([128, 2, 2])
tensor(1) tensor([0.5486, 0.4514])
torch.Size([128, 2, 2])
tensor(0) tensor([0.3449, 0.6551])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6389, 0.3611])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0375, 0.9625])
torch.Size([128, 2, 2])
tensor(0) tensor([0.4869, 0.5131])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5240, 0.4760])
torch.Size([128, 2, 2])
tensor(1) tensor([0.0935, 0.9065])
torch.Size([128, 2, 2])
tensor(0) tensor([0.8040, 0.1960])
torch.Size([128, 2, 2])
tensor(1) tensor([0.1588, 0.8412])
torch.Size([128, 2, 2])
tensor(0) tensor([0.7383, 0.2617])
torch.Size([128, 2, 2])
tensor(1) tensor([0.2704, 0.7296])
torch.Size([128, 2, 2])
tensor(0) tensor([0.2149, 0.7851])
torch.Size([128, 2, 2])
tensor(1) tensor([0.6937, 0.3063])
torch.Size([128, 2, 2])
tensor(0) tensor([0.8412, 0.1588])
torch.Size([128, 2, 2])
tensor(1) tensor([0.5313, 0.4687])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5578, 0.4422])
torch.Size([128, 2, 2])
tensor(0) tensor([0.5222, 0.4778

**^ Accuracy = 76%**

## Going to use GPU Now